In [35]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('../../data/lyrics'):
    for filename in filenames:
        print(os.path.join(dirname, filename).replace("\\","/"))


from src.progress_bar import printProgressBar

../../data/lyrics/BB-L-EN.csv
../../data/lyrics/BB-L.csv
../../data/lyrics/BB-L_invalid.json
../../data/lyrics/BB-L_invalid_updated.csv
../../data/lyrics/BB-L_raw.csv
../../data/lyrics/covid_lyrics_bb.csv
../../data/lyrics/N-BB-L-EN.csv
../../data/lyrics/N-BB-L.csv
../../data/lyrics/N-BB-L_invalid.json
../../data/lyrics/N-BB-L_invalid_updated.csv
../../data/lyrics/N-BB-L_raw.csv


### Import Raw Lyrics

In [36]:
bb_l = pd.read_csv('../../data/lyrics/N-BB-L_raw.csv', encoding='utf-8', index_col=0)
bb_l.head()


,id,artist,first_artist,song,weeks_on_chart,peak_rank,first_appearance,lyrics,url,genius_id,genius_primary_artist,genius_description,genius_annotations,genius_comments,release_date,url_first_word,artist_first_word
0,1093,2 Chainz Featuring Ariana Grande,2 Chainz,Rule The World,2,94,2019-03-16,Yeah\nUh-huh\n(Hitmaka)\n2 Chainz\n\nTop down on the Dawn like I'm used to it (Yeah)\nPrayin' he make it home like I'm used to it\nPrayin' he make...,https://genius.com/2-chainz-rule-the-world-lyrics,4327593.0,"2 Chainz, 0 songs",“Rule the World” sees 2 Chainz rapping about his respective partner. The Ariana Grande-assisted track arrives exactly a month after their “7 rings...,"[('That mean don’t call 12 on me, baby (Don’t call 12)', [['2 Chainz is originally from College Park, Georgia. Many Georgia, particularly Atlanta,...","['Who would have thought that these two artists would be so good together?', 'Ariana & 2 Chainz murdered this song', 'It’s LIT (in my inner Travis...",2019-03-01,2,2
1,1099,2 Chainz Featuring Kendrick Lamar,2 Chainz,Momma I Hit A Lick,1,100,2019-03-16,"I want it, I want it, I want it-it-it-it\n\nChilly, chilly, chilly, chilly (Yeah, what?)\nWake up, Count Chilly (What?)\nI'm up, Count Chilly (Up)...",https://genius.com/2-chainz-momma-i-hit-a-lick-lyrics,4346683.0,"2 Chainz, 0 songs",“Momma I Hit a Lick” is the first direct collaboration between 2 Chainz and Kendrick Lamar. The Pharell-produced track sees the two boasting their...,"[('I don’t take pics on a jet, lil nigga\n This shit regular, G5s all in my backyard', [['It’s common across all professions for celebrities to ta...","['Damn. He changed his voice 3 or 4 times man.', 'Finally, some new Kenny!', 'this gon be some heat!', 'Kenny came in with that asmr 21 savage ish...",2019-03-01,2,2
2,1074,2 Chainz Featuring Travis Scott,2 Chainz,Whip,1,75,2019-03-16,"Yeah\nDo it no hands, yeah, do it, no handstand\nThey really wanna keep us outside\n'Cause you know we go way too live\n\nWhippin' again, whippin'...",https://genius.com/2-chainz-whip-lyrics,4327641.0,"2 Chainz, 0 songs","In the sixth track off of 2 Chainz’s highly anticipated album Rap or Go to the League, Chainz and longtime collaborator Travis Scott deliver verse...","[('Rap or Go to the League blimp, uh', [['To announce the impending arrival of this album, 2 Chainz purchased a one million dollar blimp emblazone...","['LA FLAME X 2CHAINZ BEEN A DEADLY COMBO SINCE UPPER ECHELON', 'Travis is already the feature king of 2019', 'Wondagurl is something special 🔥', '...",2019-03-01,2,2
3,85,21 Savage,21 Savage,1.5,1,86,2019-01-05,"My earrings cost a half a ticket, I don't hear the same shit y'all niggas hear\nOn God\n\nPockets on Cheez-Its, heavy on the cheddar (Cheddar)\nRu...",https://genius.com/21-savage-15-lyrics,4155501.0,"21 Savage, 0 songs","“1.5” is a collaboration between Migos member, Offset, and 21 Savage. The two previously worked together on their collab tape, Without Warning, wi...","[('Too much drip, I’m rainin’, bought a Gucci umbrella (Umbrella)', [['In this line, 21 Savage says he has lots of “drip.” In a literal sense, dri...","['THE WORLD NEEDS “WITHOUT WARNING 2” 🙏🏽', 'if song include 21 and Offset then don’t worry it’s legendary song', 'Goddamn this goes hard!!!', 'Thi...",2018-12-21,21,21
4,36,21 Savage,21 Savage,A Lot,23,12,2019-01-05,"I love you\nTurn my headphone down a little bit, yeah\nFor so many reasons\nYeah, yeah, yeah, yeah, yeah, yeah (I do), yeah, yeah yeah yeah\nYeah,...",https://genius.com/21-savage-a-lot-lyrics,4155494.0,"21 Savage, 0 songs","On the introspective introduction to his sophomore studio album, 21 Savage raps about having “a lot” of blessings and curses in his life. Savage a...","[('[Verse 3: J. Cole]', [['Genius broke down the rhyme schemes within J. Cole’s verse on this track in an episode of the series “Check The Rhyme”:...","['J COLE CAME IN LIKE', 'J. COLE SNAPPED ON THIS TRACK WOAH 🔥', 'The collab I never thought I needed in my life :D', '21 Savage and J. C

### Import Updated Lyrics

In [37]:
bb_l_updated = pd.read_csv('../../data/lyrics/N-BB-L_invalid_updated.csv', encoding='utf-8', index_col=0)
bb_l_updated.to_excel('../../data/output/N-BB-L-invalid_updated.xlsx')
bb_l_updated = bb_l_updated.fillna(" ")
bb_l_updated.tail()


,id,artist,first_artist,song,weeks_on_chart,peak_rank,first_appearance,lyrics,url,genius_id,genius_primary_artist,genius_description,genius_annotations,genius_comments,release_date,url_first_word,artist_first_word
1316,20,"benny blanco, Halsey & Khalid","benny blanco, Halsey",Eastside,52,9,2019-01-05,"Uh\nYeah, yeah\n\nWhen I was young, I fell in love\nWe used to hold hands, man, that was enough (Yeah)\nThen we grew up, started to touch\nUsed to...",https://genius.com/Benny-blanco-halsey-and-khalid-eastside-lyrics,3828011,"​benny blanco, Halsey & Khalid",“Eastside” focuses on the relationship of two people who fall in love. It follows them as they go through their relationship and as they reminisce...,"[('In the city where the sun don’t set', [['Wordplay is used here. The east is where the sun rises from, but you can, of course, see the sunset on...","['Halsey breaking hearts with a BOP!!! 🔥🔥🔥', 'What a collaboration.', 'This will be 2018’s top summer song, save my words', 'The collab we didn’t ...",2018-07-12,benny,benny
1317,1092,"benny blanco, Tainy, Selena Gomez & J Balvin","benny blanco, Tainy, Selena Gomez",I Can't Get Enough,5,66,2019-03-16,"Crazy\nI like that, you like that, so let's be crazy\nThe contact, impact, I want that daily\nOur breath getting deeper, deeper, lately\nI like th...",https://genius.com/Benny-blanco-tainy-selena-gomez-and-j-balvin-i-cant-get-enough-lyrics,4325305,"‌benny blanco, Tainy, Selena Gomez & J Balvin",“I Can’t Get Enough” is a bilingual electronic song by producers Benny Blanco and Tainy with vocals by American singer Selena Gomez and Colombian ...,"[('Dice que no le es suficiente (Wuh)\n ’Tá mal de la mente (Yah)', [['Jhay Cortez, who is an amazing artist and songwriter, sat down with me and ...","['ISSA BOP who else missed her voice 😍👏', 'another banger 🔥🔥🔥🔥\nhttps://goo.gl/images/bgHQhQ', 'ohh Selena…😍', 'I cant get enough of this song😍😍💎💎...",2019-02-28,benny,benny
1318,3890,blackbear,blackbear,Hot Girl Bummer,42,11,2019-09-28,"Fuck you, and you, and you\nI hate your friends and they hate me too\nI'm through, I'm through, I'm through\nThis that hot girl bummer anthem\nTur...",https://genius.com/Blackbear-hot-girl-bummer-lyrics,4708753,​blackbear,"“hot girl bummer,” blackbear’s first single since his fifth studio album, ANONYMOUS, was released alongside a low-budget music video directed by G...","[('They can’t fit me in a Trojan', [['Here the speaker trots out one of the most common excuses for not wearing a condom: the old, self-flattering...","['that cover art is giving me ‘digital druglord’ vibes', 'shit already slaps', 'New song for me to listen on repeat till I’m sick of it loll. This...",2019-08-23,blackbear,blackbear
1319,3693,for KING & COUNTRY,for KING,God Only Knows,1,94,2019-09-14,"Wide awake while the world is sound asleepin'\nToo afraid of what might show up while you're dreamin'\nNobody, nobody, nobody sees you\nNobody, no...",https://genius.com/For-king-and-country-god-only-knows-lyrics,3789374,​for KING & COUNTRY,?,"[('God only knows what you’ve been through\n God only knows what they say about you\n God only knows how it’s killing you', [['This is referencing...","['I love this song!', 'Donna, I think what you are wanting to know is actually the post-chorus lyrics. I wondered the same thing and couldn’t quit...",2018-07-27,for,for
1321,6822,twenty one pilots,twenty one pilots,Level Of Concern,11,23,2020-04-25,"Need you, tell me\nNeed you, tell me\n\nPanic on the brain, world has gone insane\nThings are starting to get heavy, mm\nI can't help but think I ...",https://genius.com/Twenty-one-pilots-level-of-concern-lyrics,5428841,​twenty one pilots,“Level of Concern” is a “simple but hopeful” song about Tyler’s thoughts concerning the effects the coronavirus made on society and the music indu...,"[('Things are starting to get heavy, mm\n I can’t help but think I haven’t felt this way\n Since I asked you to go steady', [['KISS 107.1 Cincinna...","['', 'WHO IS MICHAEL AND JUL

### Update Invalid Lyrics

In [38]:
bb_l.iloc[97]

id                                                                                                                                                                        9707
artist                                                                                                                      Ariana Grande Feat. Doja Cat & Megan Thee Stallion
first_artist                                                                                                                                      Ariana Grande Feat. Doja Cat
song                                                                                                                                                                     34+35
weeks_on_chart                                                                                                                                                               8
peak_rank                                                                                                                    

In [39]:
bb_l.update(bb_l_updated)
bb_l.iloc[97]

id                                                                                                                                                                      9707.0
artist                                                                                                                      Ariana Grande Feat. Doja Cat & Megan Thee Stallion
first_artist                                                                                                                                      Ariana Grande Feat. Doja Cat
song                                                                                                                                                                     34+35
weeks_on_chart                                                                                                                                                             8.0
peak_rank                                                                                                                    

Remove Unwated Strings

In [40]:
def clean_string(string):
    i = string.find('EmbedShare URLCopyEmbedCopy')
    if i == 0:
        return string
    i -=1
    while i > 0 and string[i].isdigit():
        i -=1
    return string[:i+1]

bb_l['lyrics'] = bb_l['lyrics'].apply(lambda string: clean_string(string))


Show Shape and Collumns

In [41]:
print(bb_l.shape)
print(bb_l.columns)

(1322, 17)
Index(['id', 'artist', 'first_artist', 'song', 'weeks_on_chart', 'peak_rank',
       'first_appearance', 'lyrics', 'url', 'genius_id',
       'genius_primary_artist', 'genius_description', 'genius_annotations',
       'genius_comments', 'release_date', 'url_first_word',
       'artist_first_word'],
      dtype='object')


### Language Detector

In [42]:
import spacy
from spacy.language import Language
from langdetect import DetectorFactory
from spacy_langdetect import LanguageDetector

@Language.factory("language_detector")
def create_language_detector(nlp, name):
   return LanguageDetector(language_detection_function=None)
#

In [43]:
nlp = spacy.load("en_core_web_lg")
nlp.add_pipe('language_detector')

In [45]:
#document level
def detect_language(text, nlp):
    DetectorFactory.seed = 0
    if type(text) == str:
        doc = nlp(text)
        result = doc._.language['language'], doc._.language['score']
    else:
        result = {'-': -1}
    return result


# Retrieve data from cache to save time
try:
    languages = pd.read_csv('../../data/cache/lyrics_preparation_language_new.csv', index_col=0)
    scores = pd.read_csv('../../data/cache/lyrics_preparation_language_score_new.csv', index_col=0)
except FileNotFoundError:
    languages = []
    scores = []
    l = len(bb_l)

    printProgressBar(0, l+1, prefix = 'Progress:', suffix = 'Complete', length = 50)
    for i in range(0,l):
        language, score = detect_language(bb_l.loc[i, 'lyrics'], nlp)
        languages.append(language)
        scores.append(score)
        printProgressBar(i+1, l, prefix = 'Progress:', suffix = 'Complete', length = 50)

bb_l['language'] = languages
bb_l['language_score'] = scores
bb_l['language'].to_csv('../../data/cache/lyrics_preparation_language_new.csv')
bb_l['language_score'].to_csv('../../data/cache/lyrics_preparation_language_score_new.csv')

Progress: |██████████████████████████████████████████████████| 100.0% Complete


Lyrics Length and Word Count

In [47]:
bb_l['length'] = bb_l['lyrics'].apply(lambda string: len(str(string)))
bb_l['word_count'] = bb_l['lyrics'].apply(lambda string: len(str(string).split()))

In [46]:
bb_l.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1322 entries, 0 to 1321
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     1322 non-null   float64
 1   artist                 1322 non-null   object 
 2   first_artist           1322 non-null   object 
 3   song                   1322 non-null   object 
 4   weeks_on_chart         1322 non-null   float64
 5   peak_rank              1322 non-null   float64
 6   first_appearance       1322 non-null   object 
 7   lyrics                 1322 non-null   object 
 8   url                    1322 non-null   object 
 9   genius_id              1322 non-null   float64
 10  genius_primary_artist  1322 non-null   object 
 11  genius_description     1321 non-null   object 
 12  genius_annotations     1322 non-null   object 
 13  genius_comments        1322 non-null   object 
 14  release_date           1305 non-null   object 
 15  url_

### Export

In [ ]:
bb_l = bb_l.rename(columns={'id': 'billboard_id'})
bb_l['billboard_id'] = bb_l['billboard_id'].astype(int)
bb_l.sort_values(by='billboard_id', inplace=True)
bb_l.reset_index(inplace=True, drop=True)
bb_l['lyrics_id'] = bb_l.index
bb_l = bb_l[['billboard_id', 'lyrics_id', 'artist', 'first_artist', 'song',
                 'weeks_on_chart', 'peak_rank', 'genius_id', 'lyrics', 'url', 'length',
                 'word_count', 'language', 'language_score', 'first_appearance',
                 'genius_primary_artist', 'genius_description', 'genius_annotations',
                 'genius_comments', 'release_date']]

bb_l

In [ ]:
bb_l.to_excel('../../data/output/N-BB-L.xlsx')
bb_l.to_csv('../../data/lyrics/N-BB-L.csv')